# EDA

- Joseph 2022-09-01

In [62]:
import os
import subprocess
import operator
from pathlib import Path
from datetime import datetime

from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Firstly, let's count the number of files:

In [23]:
csv_files = os.listdir("data")
print(f"There are {len(csv_files)} csv files.")

There are 463 csv files.


Then, we want to know the begin time and end time,
i.e., the historical period, of the each pair/cryptocurrency.
We create a variable `data_timerange` to store
this information:

In [24]:
data_timerange = {
    "pair_name": [],
    "base": [],
    "quote": [],
    "start_time": [],
    "end_time": []
}
for csv_file in tqdm(csv_files):
    pair_name = csv_file.replace(".csv", "")
    if "-" in pair_name:
        base_coin, quote_coin = pair_name.split("-")
    else:
        base_coin = pair_name[:-3]
        quote_coin = pair_name[-3:]

    # Use linux command `awk` to fast analyze the data
    cmd = "awk -F ',' 'NR==2 {print $1} END {print $1}' data/" + csv_file
    p = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, shell=True)
    dt = p.stdout.read()[:-1].decode("utf-8")
    if dt.count("\n") != 1:
        continue
    start_str, end_str = dt.split("\n")
    start_dt = datetime.fromtimestamp(int(start_str)/1000)
    end_dt = datetime.fromtimestamp(int(end_str)/1000)

    # Use pandas to analyze the data (less efficient)
    # df = pd.read_csv(f"data/{csv_file}")
    # if df.empty:
    #     continue
    # start_dt = datetime.fromtimestamp(int(df.iloc[0]["time"]/1000))
    # end_dt = datetime.fromtimestamp(int(df.iloc[-1]["time"]/1000))

    data_timerange["pair_name"].append(pair_name)
    data_timerange["base"].append(base_coin)
    data_timerange["quote"].append(quote_coin)
    data_timerange["start_time"].append(start_dt)
    data_timerange["end_time"].append(end_dt)
data_timerange = pd.DataFrame(data_timerange)

100%|██████████| 463/463 [05:32<00:00,  1.39it/s]


We can peek at the result, and save it.

In [26]:
data_timerange.to_csv("EDA_results/data_timerange.csv")
data_timerange.head()

,pair_name,base,quote,start_time,end_time
0,xrpusd,xrp,usd,2017-05-20 01:57:00,2022-08-10 19:26:00
1,ccdust,ccd,ust,2022-02-10 18:04:00,2022-08-11 08:22:00
2,planets-ust,planets,ust,2021-08-11 04:08:00,2021-08-29 21:14:00
3,velo-usd,velo,usd,2021-08-11 09:28:00,2021-08-29 05:21:00
4,batust,bat,ust,2021-09-14 21:02:00,2022-08-10 19:24:00


To develop a meaningful trading strategy, sufficient
historical data must be available.
Since many of the cryptocurrencies either emerge late
or fail to survive, they are not qualified as the
trading instruments. Here we are only interested in
those that have a complete history from 2020-01-01 to 2022-08-10.

We are going to use 2 years' historical data (
2020-01-01 - 2021-12-31) as the training dataset to develop our
strategy; while leaving the 8 months' data (2022-01-01 -
2022-08-10) as the testing dataset. We will develop a
strategy denominated on`USD`, therefore we
only consider`USD`as the quote currency.

We can find out the number of cryptocurrency that satisfies
these criteria is 73:

In [27]:
data_timerange_selected = data_timerange[
    (data_timerange.start_time <= "2020-01-01")
    & (data_timerange.end_time >= "2022-08-10")
    & (data_timerange.quote == "usd")
].reset_index(inplace=False)
print(f"Number of pairs that are qualified: {data_timerange_selected.shape[0]}")

Number of pairs that are qualified: 73


Among the 73 pairs of cryptocurrencies, they are not being
traded equally actively. We should expect some of them
are highly liquid, while some of them are traded less
frequently. A security that is traded less frequently
means there might not be sufficient historical data for
us to develop a meaningful strategy (as there could
be many`NAN`values, i.e., missing values, in the dataset).

Hence we need to find out those with high liquidity.
To this end, let's measure the average number of records
within these selected 73 pairs:


In [64]:
start_dt = datetime(2020, 1, 1)
end_dt = datetime(2022, 8, 10)
num_days = (end_dt - start_dt).days
start_ts = start_dt.timestamp() * 1000
end_ts = end_dt.timestamp() * 1000
data_timerange_selected_count = {}
for i, row in tqdm(data_timerange_selected.iterrows()):
    cmd = (
        f"awk '$1 >= {start_ts} && $1 <= {end_ts} "
        "{ count++ } END { print count }' "
        f"data/{row.pair_name}.csv"
    )
    p = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, shell=True)
    res = p.stdout.read()[:-1].decode("utf-8")
    # Average number of 1min bar in 1 hour
    avg_num_bars = float(res) / num_days / 24.0
    data_timerange_selected_count[row.pair_name] = avg_num_bars

sorted_data_timerange_selected_count = dict(
    sorted(data_timerange_selected_count.items(),
           key=operator.itemgetter(1),
           reverse=True)
)
sorted_data_timerange_selected_count

73it [02:32,  2.09s/it]


{'btcusd': 59.387036064425764,
 'ethusd': 57.084252450980394,
 'xrpusd': 53.662421218487395,
 'leousd': 50.471288515406165,
 'ustusd': 49.32274159663866,
 'ltcusd': 46.442970938375346,
 'eosusd': 38.16032037815126,
 'trxusd': 32.464635854341736,
 'iotusd': 29.858674719887954,
 'omgusd': 28.966736694677873,
 'dshusd': 28.289522058823533,
 'xtzusd': 28.125919117647058,
 'neousd': 26.996104691876752,
 'zecusd': 26.854429271708682,
 'xlmusd': 25.863489145658264,
 'etcusd': 25.144826680672267,
 'gtxusd': 24.986607142857142,
 'xmrusd': 23.228203781512605,
 'bsvusd': 23.174632352941178,
 'algusd': 22.860294117647058,
 'vsyusd': 19.907256652661065,
 'zrxusd': 19.524903711484594,
 'atousd': 19.069152661064425,
 'uosusd': 18.80400910364146,
 'ampusd': 15.793723739495798,
 'bttusd': 12.78532037815126,
 'clousd': 12.084033613445378,
 'daiusd': 11.967612044817926,
 'iqxusd': 11.579262955182074,
 'udcusd': 11.091123949579831,
 'vetusd': 10.877494747899158,
 'eutusd': 10.664259453781513,
 'xvgusd': 1

{'btcusd': 1425.2888655462184,
 'ethusd': 1370.0220588235295,
 'xrpusd': 1287.8981092436975,
 'leousd': 1211.310924369748,
 'ustusd': 1183.7457983193278,
 'ltcusd': 1114.6313025210084,
 'eosusd': 915.8476890756302,
 'trxusd': 779.1512605042017,
 'iotusd': 716.6081932773109,
 'omgusd': 695.2016806722689,
 'dshusd': 678.9485294117648,
 'xtzusd': 675.0220588235294,
 'neousd': 647.906512605042,
 'zecusd': 644.5063025210084,
 'xlmusd': 620.7237394957983,
 'etcusd': 603.4758403361344,
 'gtxusd': 599.6785714285714,
 'xmrusd': 557.4768907563025,
 'bsvusd': 556.1911764705883,
 'algusd': 548.6470588235294,
 'vsyusd': 477.77415966386553,
 'zrxusd': 468.5976890756302,
 'atousd': 457.65966386554624,
 'uosusd': 451.296218487395,
 'ampusd': 379.04936974789916,
 'bttusd': 306.8476890756302,
 'clousd': 290.01680672268907,
 'daiusd': 287.2226890756302,
 'iqxusd': 277.9023109243698,
 'udcusd': 266.18697478991595,
 'vetusd': 261.0598739495798,
 'eutusd': 255.9422268907563,
 'xvgusd': 251.7563025210084,
 '

In [65]:
res

'129857'

In [41]:
start_ts

1577808000.0

Prepare data files for backtest:

In [34]:
Path("clean_data/k_line/K_1M").mkdir(parents=True, exist_ok=True)

In [35]:
for i, row in tqdm(data_universe.iterrows()):
    df = pd.read_csv(f"data/{row.pair_name}.csv")
    df["time_key"] = df.time.apply(lambda x: datetime.fromtimestamp(int(x/1000)))
    df["date"] = df.time_key.apply(lambda x: x.date())
    df = df[
        (df.time_key >= "2020-01-01")
        & (df.time_key <= "2022-08-10")
    ]
    # print(f"{i}. {row.pair_name}: {df.shape}")
    for date_ in df.date.unique():
        df_daily = df[df.date == date_][
            ["time_key", "open", "high", "low", "close", "volume"]
        ].reset_index(drop=True)
        date_str = date_.strftime("%Y-%m-%d")
        Path(f"clean_data/k_line/K_1M/{row.pair_name}").mkdir(parents=True, exist_ok=True)
        df_daily.to_csv(f"clean_data/k_line/K_1M/{row.pair_name}/{date_str}.csv",
                        index=False)

NameError: name 'data_universe' is not defined

We want to see the active trading hours of each coin.

In [ ]:
num_bars_in_1hr = {}
for i, row in tqdm(data_timerange_selected.iterrows()):
    # if i > 7:
    #     break
    if i % 4 == 0:
        fig, axes = plt.subplots(
            nrows=4, #data_universe.shape[0],
            ncols=1,
            sharex=True
        )
    df = pd.read_csv(f"data/{row.pair_name}.csv")
    df["time_key"] = df.time.apply(lambda x: datetime.fromtimestamp(int(x/1000)))
    df["date"] = df.time_key.apply(lambda x: x.date())
    df["hour"] = df.time_key.apply(lambda x: x.hour)
    df = df[
        (df.time_key >= "2020-01-01")
        & (df.time_key <= "2022-08-10")
    ]
    num_days = len(df.date.unique())
    # Average number of bars in that particular hour
    df1 = df.groupby("hour").count().time_key / num_days
    df1.plot(ax=axes[i%4], title=f"{row.pair_name}")
    num_bars_in_1hr[row.pair_name] = {
        "max": df1.max(),
        "min": df1.min(),
        "mean": df1.mean(),
        "median": df1.median(),
        "std": df1.std()
    }
num_bars_in_1hr = pd.DataFrame(num_bars_in_1hr).T
num_bars_in_1hr = num_bars_in_1hr.sort_values(by="min", ascending=False)
num_bars_in_1hr

We can see that `BTC` and `ETH` are trading most actively; while
coins such as `LYM` and `PAS` are almost inactive. The table shows
that there are 59 bars (1-min OHLCV) in every hour for `BTC`, and 57
bars for `ETH`, which indicates high liquidity for these two
cryptocurrencies. The less this number becomes, the more illiquid
it means. Let's list the top 10 coins:


In [ ]:
num_bars_in_1hr.head(10)

As we can see, only the first 6 pairs are sufficient liquid (with
more than 45 bars in an hour). There is a big gap between `LTC`
and `EOS`. In order to guarantee the liquidity of trading
instruments, we limit our trading universe to the top 6 cryptocurrency
pairs, i.e., `BTCUSD`, `ETHUSD`, `XRPUSD`, `LEOUSD`, `USTUSD`,
and `LTCUSD`.

We can take a closer look at these coins:

In [ ]:
pairs = {
    "btcusd": None,
    "ethusd": None,
    "xrpusd": None,
    "leousd": None,
    "ustusd": None,
    "ltcusd": None
}
for i, pair_name in enumerate(pairs):
    print(i, i%2, pair_name)
    if i % 2 == 0:
        fig, axes = plt.subplots(
            nrows=2, #data_universe.shape[0],
            ncols=1,
            sharex=True
        )
    df = pd.read_csv(f"data/{pair_name}.csv")
    df["time_key"] = df.time.apply(lambda x: datetime.fromtimestamp(int(x/1000)))
    df["date"] = df.time_key.apply(lambda x: x.date())
    df["hour"] = df.time_key.apply(lambda x: x.hour)
    df = df[
        (df.time_key >= "2020-01-01")
        & (df.time_key <= "2022-08-10")
    ]
    pairs[pair_name] = df.copy()
    num_days = len(df.date.unique())
    # Average number of bars in that particular hour
    df1 = df.groupby("hour").count().time_key / num_days
    df1.plot(ax=axes[i%2], title=f"{pair_name}")

As we can see, all six pairs have similar active trading
hours. The market is relatively silent from 9 am to 4 pm.
This may indicate that the investors of different cryptocurrencies
are likely the same group of people.

Let's look at the price movements of these cryptocurrencies:

In [ ]:
for i, pair_name in enumerate(pairs):
    if i % 2 == 0:
        fig, axes = plt.subplots(
            nrows=2, #data_universe.shape[0],
            ncols=1,
            sharex=True
        )
    df = pairs[pair_name]
    df[["time_key", "close"]].set_index("time_key").plot(
        ax=axes[i%2],
        title=pair_name)

We can see that `BTC`, `ETH`, `XRP`, and `LTC` show similarity
in price movement, but `LEO` and `UST` differ from the others.
We can see it more clear in a correlation heatmap:

In [ ]:
all_pairs = None
for pair_name in pairs:
    df = pairs[pair_name][["time_key", "close"]].set_index("time_key")
    print(pair_name, df.shape)
    # if all_pairs is None:
    #     all_pairs = df.copy()
    # else:
    #     all_pairs = all_pairs.merge(df, how="outer")